# G1 scraping
This notebook aims to build a scaper of the news in https://g1.globo.com/

## Setup

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd

# Path to geckodriver executable
geckodriver_path = '/snap/bin/firefox.geckodriver'


s = Service(executable_path=geckodriver_path)
# Create a Firefox webdriver instance
# opens a window
driver = webdriver.Firefox(service=s)

## Start

In [2]:
driver.get('https://g1.globo.com/')

In [3]:
# scrolling the page

scrolls = 10

current_height = driver.execute_script("return document.body.scrollHeight")

for i in range(scrolls):
    # scroll to the end of the page
    driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")

    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # if not autoscoll
    if new_height == current_height:
        # click the 'Veja mais' button
        driver.find_element(By.CSS_SELECTOR, value='.load-more > a:nth-child(1)').click()
        
        # repeat the scroll
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
        current_height = driver.execute_script("return document.body.scrollHeight")
    else:
        current_height = new_height
        
    print(f'Scroll {i + 1}, height={current_height}')
    
    # time for the page to load
    sleep(4)

Scroll 1, height=8133
Scroll 2, height=10593
Scroll 3, height=13730
Scroll 4, height=17030
Scroll 5, height=20165
Scroll 6, height=23462
Scroll 7, height=19678
Scroll 8, height=23007
Scroll 9, height=22003
Scroll 10, height=25569


## Scraping the data

In [4]:
html_source = driver.page_source

In [5]:
soup = BeautifulSoup(html_source, 'lxml')

In [6]:
content_blocks = soup.find_all('div', class_ = '_evg')
len(content_blocks)

12

In [12]:
titles = []
times = []
sections = []
headers = []
resumes = []

for block in content_blocks:
    news_list = block.find_all('div', class_ = 'feed-post-body')
    for news in news_list:
        title = news.find('a', class_ = 'feed-post-link gui-color-primary gui-color-hover')
        titles.append(title.text)
        
        header = news.find('span', 'feed-post-header-chapeu')
        headers.append(None if header is None else header.text)

        time = news.find('span', 'feed-post-datetime')
        times.append(None if time is None else time.text)

        section = news.find('span', 'feed-post-metadata-section')
        sections.append(None if section is None else section.text)

        resume = news.find('div', class_='feed-post-body-resumo')
        resumes.append(None if resume is None else resume.text)

        

data = pd.DataFrame({
    'Title': titles,
    'Time': times,
    'Section': sections,
    'Header': headers,
    'Resume': resumes
})

data
        

,Title,Time,Section,Header,Resume
0,PF descobre dispositivos ilegais para roubar d...,Há 1 hora,Política,Sistema clandestino,Varredura encontrou instalações clandestinas e...
1,'Abin paralela' de Bolsonaro espionou ministro...,Há 3 horas,Política,Monitoramento ilegal,None
2,VALDO: áudio de Ramagem e Bolsonaro sobre Fláv...,Há 2 horas,Blog do Valdo Cruz,None,None
3,"Carne, remédios, cerveja, carro: o que pode mu...",Há 7 horas,Economia,Pagamento de impostos,Deputados decidiram que armas não terão tribut...
4,Reforma prevê imposto zero para quase 400 remé...,Há 7 horas,Saúde,De viagra a remédio para asma,None
...,...,...,...,...,...
123,Abin foi usada para produzir prova a favor de ...,Há 5 horas,Política,None,"Sob comando de Alexandre Ramagem, agentes teri..."
124,Abin paralela: os núcleos do esquema ilegal de...,Há 5 horas,Política,None,"Investigações apontam que policiais, servidore..."
125,Abin paralela: o que dizem os investigados pel...,Há 6 horas,Política,None,Operação investiga se governo Bolsonaro usou s...
126,"Ana Maria Braga, Ludmilla e outros famosos com...",Há 6 horas,Pop & Arte,None,Cantora recebeu o apoio de amigos e amigas cel...


In [8]:
titles

['PF descobre dispositivos ilegais para roubar dados do INSS',
 "'Abin paralela' de Bolsonaro espionou ministros do STF, políticos e jornalistas, diz PF",
 'VALDO: áudio de Ramagem e Bolsonaro sobre Flávio preocupa aliados',
 'Carne, remédios, cerveja, carro: o que pode mudar na reforma tributária',
 'Reforma prevê imposto zero para quase 400 remédios; veja lista',
 'Justiça dá 48 horas para líder da Bola de Neve entregar armas',
 'Palmeiras x Atlético-GO',
 'Explosão interrompe entrevista de motorista de ônibus em chamas',
 'VÍDEO flagra momento em que avião da Latam bate cauda ao decolar',
 "'Mau perdedor': imprensa critica Macron por ignorar resultado da eleição",
 "Morte de Shelley Duvall foi anunciada pelo marido: 'Agora está livre'",
 "Shelley Duvall, atriz americana de 'O iluminado' e 'Popeye', morre aos 75 anos",
 '3 animais que conseguem detectar doenças em humanos',
 'Qual sua relação com a bebida alcóolica? Faça o teste e descubra',
 'Brasileiros já compraram mais de 57 mil 